In [33]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer

In [76]:
with open ('/Users/dmitrywer/downloads/results-gpt4o-mini.csv', 'r') as file:
    df = pd.read_csv(file)
    df = df.iloc[:300]

In [22]:
df

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp
...,...,...,...,...,...
295,An alternative way to load the data using the ...,Above users showed how to load the dataset dir...,8d209d6d,What is an alternative way to load the data us...,machine-learning-zoomcamp
296,You can directly download the dataset from Git...,Above users showed how to load the dataset dir...,8d209d6d,How can I directly download the dataset from G...,machine-learning-zoomcamp
297,You can fetch data for homework using the `req...,Above users showed how to load the dataset dir...,8d209d6d,Could you share a method to fetch data for hom...,machine-learning-zoomcamp
298,If the status code is 200 when downloading dat...,Above users showed how to load the dataset dir...,8d209d6d,What should I do if the status code is 200 whe...,machine-learning-zoomcamp


In [29]:
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

You try to use a model that was created with version 3.0.0.dev0, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [30]:
answer_llm = df.iloc[0].answer_llm
embedding_model.encode(answer_llm)[0]

-0.42244658

In [32]:
evaluations = []

for i, row in df.iterrows():
    llm_embedding = embedding_model.encode(row['answer_llm'])
    orig_embedding = embedding_model.encode(row['answer_orig'])
    evaluations.append(llm_embedding.dot(orig_embedding))

In [34]:
percentile_75 = np.percentile(evaluations, 75)

In [36]:
percentile_75

31.67430353164673

## Lets normilize vectors

In [41]:
def normilize_vector(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

In [42]:
evaluations = []

for i, row in df.iterrows():
    llm_embedding = embedding_model.encode(row['answer_llm'])
    orig_embedding = embedding_model.encode(row['answer_orig'])

    llm_norm = normilize_vector(llm_embedding)
    orig_norm = normilize_vector(orig_embedding)

    evaluations.append(llm_norm.dot(orig_norm))

In [43]:
percentile_75 = np.percentile(evaluations, 75)

In [44]:
percentile_75

0.8362347632646561

## Rouge

In [54]:
df_short = df[df['document'] == '5170565b']

In [55]:
df_short

,answer_llm,answer_orig,document,question,course
10,"Yes, all sessions are recorded, so if you miss...","Everything is recorded, so you won’t miss anyt...",5170565b,Are sessions recorded if I miss one?,machine-learning-zoomcamp
11,"Yes, you can ask your questions in advance if ...","Everything is recorded, so you won’t miss anyt...",5170565b,Can I ask questions in advance if I can't atte...,machine-learning-zoomcamp
12,"If you miss a session, don't worry! Everything...","Everything is recorded, so you won’t miss anyt...",5170565b,How will my questions be addressed if I miss a...,machine-learning-zoomcamp
13,"Yes, there is a way to catch up on a missed se...","Everything is recorded, so you won’t miss anyt...",5170565b,Is there a way to catch up on a missed session?,machine-learning-zoomcamp
14,"Yes, you can still interact with instructors a...","Everything is recorded, so you won’t miss anyt...",5170565b,Can I still interact with instructors after mi...,machine-learning-zoomcamp


In [56]:
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(df_short['answer_llm'], df_short['answer_orig'])[0]

In [61]:
scores['rouge-1']['f']

0.45454544954545456

In [47]:
df

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp
...,...,...,...,...,...
295,An alternative way to load the data using the ...,Above users showed how to load the dataset dir...,8d209d6d,What is an alternative way to load the data us...,machine-learning-zoomcamp
296,You can directly download the dataset from Git...,Above users showed how to load the dataset dir...,8d209d6d,How can I directly download the dataset from G...,machine-learning-zoomcamp
297,You can fetch data for homework using the `req...,Above users showed how to load the dataset dir...,8d209d6d,Could you share a method to fetch data for hom...,machine-learning-zoomcamp
298,If the status code is 200 when downloading dat...,Above users showed how to load the dataset dir...,8d209d6d,What should I do if the status code is 200 whe...,machine-learning-zoomcamp


In [81]:
from rouge import Rouge
rouge_scorer = Rouge()

scores = rouge_scorer.get_scores(df_short['answer_llm'], df_short['answer_orig'])[0]

In [82]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [85]:
total_f_score = []
for s in scores:
    total_f_score.append(scores[f'{s}']['f'])

In [86]:
sum(total_f_score) / len(total_f_score)

0.35490034990035496

## Compute F score for each record

In [95]:
rouge_1 = []
rouge_2 = []
rouge_l = []

for item in df.itertuples():
    scores = rouge_scorer.get_scores(item.answer_llm, item.answer_orig)[0]
    scores
    rouge_1.append(scores['rouge-1']['f'])
    rouge_2.append(scores['rouge-2']['f'])
    rouge_l.append(scores['rouge-l']['f'])

In [96]:
sum(rouge_2) / len(rouge_2)

0.2069650198342332

In [97]:
df_scores = {
    'rouge_1': rouge_1,
    'rouge_2': rouge_2,
    'rouge_l': rouge_l
}